In [49]:
import numpy as np

import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
pd.set_option('expand_frame_repr', True)
import math

from os import listdir

In [55]:
files_to_parse = listdir('raw_files/2018-daily-until-0126')
column_names = [
'GLOBALEVENTID','SQLDATE','MonthYear','Year','FractionDate','Actor1Code','Actor1Name','Actor1CountryCode','Actor1KnownGroupCode	',
'Actor1EthnicCode','Actor1Religion1Code','Actor1Religion2Code','Actor1Type1Code','Actor1Type2Code','Actor1Type3Code',
'Actor2Code','Actor2Name','Actor2CountryCode','Actor2KnownGroupCode','Actor2EthnicCode','Actor2Religion1Code',
'Actor2Religion2Code','Actor2Type1Code','Actor2Type2Code','Actor2Type3Code','IsRootEvent','EventCode','EventBaseCode',
'EventRootCode','QuadClass','GoldsteinScale','NumMentions','NumSources','NumArticles','AvgTone','Actor1Geo_Type',
'Actor1Geo_FullName','Actor1Geo_CountryCode','Actor1Geo_ADM1Code','Actor1Geo_Lat','Actor1Geo_Long','Actor1Geo_FeatureID',
'Actor2Geo_Type','Actor2Geo_FullName','Actor2Geo_CountryCode','Actor2Geo_ADM1Code','Actor2Geo_Lat','Actor2Geo_Long',
'Actor2Geo_FeatureID','ActionGeo_Type','ActionGeo_FullName','ActionGeo_CountryCode','ActionGeo_ADM1Code','ActionGeo_Lat',
'ActionGeo_Long','ActionGeo_FeatureID','DATEADDED','SOURCEURL']

columns_to_keep = ['SQLDATE', 'IsRootEvent', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'AvgTone']
#optional columns later to try with; Actor1Code, Actor2Code, Actor1Geo_ADM1Code, Actor2Geo_ADM1Code

In [56]:
files_to_parse[:10]

['20180101.export.CSV',
 '20180102.export.CSV',
 '20180103.export.CSV',
 '20180104.export.CSV',
 '20180105.export.CSV',
 '20180106.export.CSV',
 '20180107.export.CSV',
 '20180108.export.CSV',
 '20180109.export.CSV',
 '20180110.export.CSV']

In [58]:
%%capture
hourly_events = pd.DataFrame({})
for file in files_to_parse:
    df = pd.read_csv('raw_files/2018-daily-until-0126/{file}'.format(file=file), header=None, sep="\t", names=column_names)
    df = df.set_index('GLOBALEVENTID')[columns_to_keep]
    splitDf = pd.DataFrame({})
    #This splits the dataframe into 24 equal parts sequentially, and marks which hour each piece belongs to
    for g, subdf in df.groupby(np.arange(len(df)) // (df.shape[0] / 24)):
        subdf['Hour'] = str(math.ceil(g)) if math.ceil(g) > 9 else ('0' + str(math.ceil(g)))
        splitDf = splitDf.append(subdf)
        
    df = splitDf
    df['EventRootCode'] = df['EventRootCode'].astype(pd.api.types.CategoricalDtype(categories= [i for i in range(1,21)]))
    df['GoldsteinScale'] = df['GoldsteinScale'] /10
    df['AvgTone'] = df['AvgTone'] /100
    
    df = pd.get_dummies(df, columns=['QuadClass', 'EventRootCode', 'IsRootEvent'], prefix=['QuadClass', 'EventRootCode', 'RootEvent_TF'])
    df = df.groupby(['Hour'], axis=0).mean().reset_index()
    
    df['DateTime'] = str(file[:4]) + '-' + str(file[4:6]) + '-' + str(file[6:8]) + ' ' + df['Hour'] + ':00'
    df = df.drop(['SQLDATE', 'Hour'], 1).set_index('DateTime')
    
    hourly_events = hourly_events.append(df)
    del df

In [59]:
hourly_events.shape

(624, 28)

In [34]:
hourly_events.columns

Index(['GoldsteinScale', 'AvgTone', 'QuadClass_1', 'QuadClass_2',
       'QuadClass_3', 'QuadClass_4', 'EventRootCode_1', 'EventRootCode_2',
       'EventRootCode_3', 'EventRootCode_4', 'EventRootCode_5',
       'EventRootCode_6', 'EventRootCode_7', 'EventRootCode_8',
       'EventRootCode_9', 'EventRootCode_10', 'EventRootCode_11',
       'EventRootCode_12', 'EventRootCode_13', 'EventRootCode_14',
       'EventRootCode_15', 'EventRootCode_16', 'EventRootCode_17',
       'EventRootCode_18', 'EventRootCode_19', 'EventRootCode_20',
       'RootEvent_TF_0', 'RootEvent_TF_1'],
      dtype='object')

In [60]:
hourly_events.to_csv('2018_hourly_until_0126.tsv', sep='\t')